# 지역별 환자 분포 공간 분석

OMOP CDM 데이터를 기반으로 환자의 지역별 분포를 시각화하는 공간분석 템플릿입니다.

**사용 라이브러리**: geopandas, folium, shapely, contextily

**분석 목표**:
- 의료 이용 패턴의 지리적 분포 파악
- 의료 접근성 사각지대 식별
- 질환별 지역 클러스터 분석

In [ ]:
# 라이브러리 설치 (최초 1회)
# !pip install geopandas folium shapely pyproj contextily mapclassify branca

import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

print('공간분석 환경 준비 완료')

In [ ]:
# 서울시 행정구역별 환자 분포 시뮬레이션 데이터 생성
# (실 데이터 연동 시 OMOP CDM person.location 활용)

districts = [
    {'name': '송파구', 'lat': 37.5145, 'lon': 127.1050, 'patients': 8420, 'diabetes': 1230, 'hypertension': 2150},
    {'name': '강남구', 'lat': 37.5172, 'lon': 127.0473, 'patients': 7850, 'diabetes': 980, 'hypertension': 1870},
    {'name': '강동구', 'lat': 37.5301, 'lon': 127.1238, 'patients': 6230, 'diabetes': 890, 'hypertension': 1560},
    {'name': '서초구', 'lat': 37.4837, 'lon': 127.0324, 'patients': 5670, 'diabetes': 720, 'hypertension': 1340},
    {'name': '광진구', 'lat': 37.5384, 'lon': 127.0822, 'patients': 4980, 'diabetes': 680, 'hypertension': 1230},
    {'name': '성동구', 'lat': 37.5634, 'lon': 127.0369, 'patients': 4520, 'diabetes': 610, 'hypertension': 1120},
    {'name': '동대문구', 'lat': 37.5744, 'lon': 127.0395, 'patients': 4120, 'diabetes': 590, 'hypertension': 1050},
    {'name': '중랑구', 'lat': 37.6063, 'lon': 127.0928, 'patients': 3870, 'diabetes': 550, 'hypertension': 980},
    {'name': '마포구', 'lat': 37.5663, 'lon': 126.9013, 'patients': 3450, 'diabetes': 420, 'hypertension': 870},
    {'name': '용산구', 'lat': 37.5326, 'lon': 126.9909, 'patients': 2890, 'diabetes': 360, 'hypertension': 720},
    {'name': '종로구', 'lat': 37.5735, 'lon': 126.9790, 'patients': 2340, 'diabetes': 290, 'hypertension': 580},
    {'name': '중구', 'lat': 37.5641, 'lon': 126.9979, 'patients': 2120, 'diabetes': 270, 'hypertension': 530},
]

df = pd.DataFrame(districts)
df['diabetes_rate'] = (df['diabetes'] / df['patients'] * 100).round(1)
df['hypertension_rate'] = (df['hypertension'] / df['patients'] * 100).round(1)
print(f'지역 {len(df)}개, 총 환자 {df.patients.sum():,}명')
df.head()

In [ ]:
# Folium 지도 시각화 — 환자 분포 버블맵
import folium
from folium.plugins import HeatMap

# 서울아산병원 중심 지도
m = folium.Map(
    location=[37.527, 127.065],  # 서울아산병원 위치
    zoom_start=12,
    tiles='cartodbpositron',
)

# 아산병원 마커
folium.Marker(
    [37.527, 127.065],
    popup='서울아산병원',
    icon=folium.Icon(color='red', icon='plus-sign'),
).add_to(m)

# 지역별 환자 수 버블
for _, row in df.iterrows():
    folium.CircleMarker(
        location=[row['lat'], row['lon']],
        radius=row['patients'] / 500,
        popup=f"{row['name']}: {row['patients']:,}명\n당뇨 {row['diabetes_rate']}%",
        color='#005BAC',
        fill=True,
        fill_opacity=0.6,
    ).add_to(m)

# 히트맵 레이어
heat_data = [[row['lat'], row['lon'], row['patients']] for _, row in df.iterrows()]
HeatMap(heat_data, name='환자 밀집도', radius=30).add_to(m)

folium.LayerControl().add_to(m)
m

In [ ]:
# 질환별 지역 분포 비교 (matplotlib)
import matplotlib.pyplot as plt
import matplotlib
matplotlib.rcParams['font.family'] = 'NanumGothic'
matplotlib.rcParams['axes.unicode_minus'] = False

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# 당뇨 비율
df_sorted = df.sort_values('diabetes_rate', ascending=True)
axes[0].barh(df_sorted['name'], df_sorted['diabetes_rate'], color='#E8594F')
axes[0].set_xlabel('당뇨 비율 (%)')
axes[0].set_title('지역별 당뇨 유병률')
for i, v in enumerate(df_sorted['diabetes_rate']):
    axes[0].text(v + 0.1, i, f'{v}%', va='center', fontsize=9)

# 고혈압 비율
df_sorted2 = df.sort_values('hypertension_rate', ascending=True)
axes[1].barh(df_sorted2['name'], df_sorted2['hypertension_rate'], color='#005BAC')
axes[1].set_xlabel('고혈압 비율 (%)')
axes[1].set_title('지역별 고혈압 유병률')
for i, v in enumerate(df_sorted2['hypertension_rate']):
    axes[1].text(v + 0.1, i, f'{v}%', va='center', fontsize=9)

plt.tight_layout()
plt.suptitle('서울아산병원 환자 지역별 질환 분포', y=1.02, fontsize=14, fontweight='bold')
plt.show()

In [ ]:
# 의료 접근성 분석 — 아산병원까지의 거리 기반
from math import radians, cos, sin, asin, sqrt

def haversine(lat1, lon1, lat2, lon2):
    """두 좌표 간 거리 (km)"""
    lat1, lon1, lat2, lon2 = map(radians, [lat1, lon1, lat2, lon2])
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    return 2 * 6371 * asin(sqrt(a))

asan_lat, asan_lon = 37.527, 127.065
df['distance_km'] = df.apply(lambda r: round(haversine(asan_lat, asan_lon, r['lat'], r['lon']), 1), axis=1)
df['patients_per_km'] = (df['patients'] / df['distance_km']).round(0)

print('=== 의료 접근성 분석 ===')
print(df[['name', 'patients', 'distance_km', 'patients_per_km']].sort_values('distance_km').to_string(index=False))
print(f'\n평균 거리: {df.distance_km.mean():.1f}km')
print(f'접근성 최고: {df.loc[df.distance_km.idxmin(), "name"]} ({df.distance_km.min()}km)')
print(f'접근성 최저: {df.loc[df.distance_km.idxmax(), "name"]} ({df.distance_km.max()}km)')